In [ ]:
import Dans_Diffraction as dif
import numpy as np
import pandas as pd

xtl = dif.Crystal(r"E:\data_processing\data\crystallographic_db\Fe3O4_1513304.cif")

xtl.Scatter.setup_scatter(scattering_type='electron', energy_kev=200, min_twotheta=0.1, max_twotheta=10)

# Get full reflection list
reflections = xtl.Scatter.get_hkl()
transmission = xtl.hkl_transmission(reflections)
q_mag = xtl.Cell.Qmag(reflections)
F2 = xtl.Scatter.electron(reflections)

# Set desired Q max in Å⁻¹
Q_max = 10.0

# Filter based on Q
mask = q_mag <= Q_max
hkl_filtered = reflections[mask]
q_filtered = q_mag[mask]
f2_filtered = F2[mask]

# Convert to DataFrame
df = pd.DataFrame(hkl_filtered, columns=["h", "k", "l"])
df["Q"] = q_filtered
df["F2"] = f2_filtered

df_nonzero = df[df["F2"] > 1e-5]

# Optional: reset index
df_nonzero = df_nonzero.reset_index(drop=True)

df_unique = df.groupby(df_nonzero["Q"].round(5), as_index=False).apply(lambda g: g.loc[g["F2"].idxmax()])
df_unique.reset_index(drop=True, inplace=True)

df_unique